In [2]:
%config IPCompleter.use_jedi = False
%pdb off
%load_ext autoreload
%autoreload 3
import sys
from copy import deepcopy
from typing import List, Dict, Optional, Union, Callable
from pathlib import Path
import pathlib
import numpy as np
import pandas as pd
import tables as tb
from copy import deepcopy
from datetime import datetime, timedelta
from attrs import define, field, Factory

# required to enable non-blocking interaction:
%gui qt5

# Jupyter interactivity:
import ipywidgets as widgets
from IPython.display import display
from pyphocorehelpers.gui.Jupyter.JupyterButtonRowWidget import JupyterButtonRowWidget

## Pho's Custom Libraries:
from pyphocorehelpers.Filesystem.path_helpers import find_first_extant_path
from pyphocorehelpers.function_helpers import function_attributes
from pyphocorehelpers.print_helpers import CapturedException

# pyPhoPlaceCellAnalysis:
# NeuroPy (Diba Lab Python Repo) Loading
from neuropy.core.session.Formats.BaseDataSessionFormats import DataSessionFormatRegistryHolder
from neuropy.core.session.Formats.Specific.BapunDataSessionFormat import BapunDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.KDibaOldDataSessionFormat import KDibaOldDataSessionFormatRegisteredClass
from neuropy.core.session.Formats.Specific.RachelDataSessionFormat import RachelDataSessionFormat
from neuropy.core.session.Formats.Specific.HiroDataSessionFormat import HiroDataSessionFormatRegisteredClass
from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update

## For computation parameters:
from neuropy.utils.result_context import IdentifyingContext
from neuropy.core.session.Formats.BaseDataSessionFormats import find_local_session_paths
from neuropy.core import Epoch

from pyphoplacecellanalysis.General.Pipeline.Stages.Loading import saveData, loadData
import pyphoplacecellanalysis.General.Batch.runBatch
from pyphoplacecellanalysis.General.Batch.runBatch import BatchRun, BatchResultDataframeAccessor, run_diba_batch, BatchComputationProcessOptions, BatchSessionCompletionHandler, SavingOptions
from pyphoplacecellanalysis.General.Pipeline.NeuropyPipeline import PipelineSavingScheme

from neuropy.core.user_annotations import UserAnnotationsManager
from pyphoplacecellanalysis.General.Batch.runBatch import SessionBatchProgress
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

from pyphocorehelpers.Filesystem.path_helpers import set_posix_windows


from pyphocorehelpers.print_helpers import CapturedException
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from pyphoplacecellanalysis.General.Batch.runBatch import BatchSessionCompletionHandler

BATCH_DATE_TO_USE = '2023-09-28' # used for filenames throught the notebook

Automatic pdb calling has been turned OFF
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
active_global_batch_result_filename=f'global_batch_result_{BATCH_DATE_TO_USE}.pkl'
debug_print = False
known_global_data_root_parent_paths = [Path(r'W:\Data'), Path(r'/media/MAX/Data'), Path(r'/Volumes/MoverNew/data'), Path(r'/home/halechr/turbo/Data'), Path(r'/nfs/turbo/umms-kdiba/Data')] # , Path(r'/home/halechr/FastData')
global_data_root_parent_path = find_first_extant_path(known_global_data_root_parent_paths)
assert global_data_root_parent_path.exists(), f"global_data_root_parent_path: {global_data_root_parent_path} does not exist! Is the right computer's config commented out above?"
## Build Pickle Path:
global_batch_result_file_path = Path(global_data_root_parent_path).joinpath(active_global_batch_result_filename).resolve() # Use Default

# try to load an existing batch result:
global_batch_run = BatchRun.try_init_from_file(global_data_root_parent_path, active_global_batch_result_filename=active_global_batch_result_filename,
						skip_root_path_conversion=False, debug_print=debug_print) # on_needs_create_callback_fn=run_diba_batch

batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    # display(batch_progress_df)
    # display(good_only_batch_progress_df)
    display(batch_progress_df)

Loading loaded session pickle file results : /media/MAX/Data/global_batch_result_2023-09-28.pkl... done.
Failure loading /media/MAX/Data/global_batch_result_2023-09-28.pkl.
creating new batch_run
local_session_names_list: ['2006-6-07_11-26-53', '2006-6-08_14-26-15', '2006-6-09_1-22-43', '2006-6-09_3-23-37', '2006-6-12_15-55-31', '2006-6-13_14-42-6']
local_session_names_list: ['2006-6-07_16-40-19', '2006-6-08_15-46-47', '2006-6-08_21-16-25', '2006-6-09_22-24-40', '2006-6-12_16-53-46', '2006-6-13_15-22-3']
local_session_names_list: ['2006-4-09_17-29-30', '2006-4-10_12-25-50', '2006-4-10_21-2-40', '2006-4-11_15-16-59', '2006-4-12_14-39-31', '2006-4-12_17-53-55', '2006-4-16_15-12-23', '2006-4-17_12-33-47', '2006-4-18_13-6-1', '2006-4-18_15-23-32', '2006-4-19_13-34-40', '2006-4-19_16-48-9', '2006-4-21_10-24-35', '2006-4-25_14-28-51', '2006-4-25_17-17-6', '2006-4-26_13-22-13', '2006-4-27_14-43-12', '2006-4-28_12-17-27', '2006-4-28_16-48-29']
local_session_names_list: ['2006-4-09_16-40-54', '

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.NOT_STARTED,None,2006-06-08 14:26:15,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.NOT_STARTED,None,2006-06-09 01:22:43,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,2006-06-09 03:23:37,0,0,0,0,False,,,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.NOT_STARTED,None,2006-06-12 15:55:31,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,2009-11-04 21:20:03,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,False,True
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/KDIBA/pin01/one/redundant,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/KDIBA/pin01/one/showclus,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/KDIBA/pin01/one/sleep,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False


# Run Batch Executions/Computations

In [4]:
# Hardcoded included_session_contexts:
included_session_contexts = [
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-08_21-16-25'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-09_22-24-40'),
    IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-10_12-25-50'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-09_16-40-54'),
    IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='two',session_name='2006-4-10_12-58-3'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_17-46-44'), # prev completed
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-02_19-28-0'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='11-03_12-3-25'),
    IdentifyingContext(format_name='kdiba',animal='pin01',exper_name='one',session_name='fet11-01_12-58-54'), # prev completed
]

included_session_batch_progress_df = batch_progress_df[np.isin(batch_progress_df['context'].values, included_session_contexts)]
with pd.option_context('display.max_rows', 10, 'display.max_columns', None):  # more options can be specified also
    included_session_batch_progress_df

# Execute Batch

In [5]:
output_included_session_contexts, output_python_scripts, output_slurm_scripts = global_batch_run.generate_batch_slurm_jobs(included_session_contexts, Path('output/generated_slurm_scripts/').resolve(), 
                                                                                                                        use_separate_run_directories=True, should_force_reload_all=False, should_perform_figure_generation_to_file=False)

In [ ]:
from pyphocorehelpers.Filesystem.open_in_system_file_manager import reveal_in_system_file_manager


button_defns = [(str(a_ctxt), lambda _: reveal_in_system_file_manager(a_script)) for a_ctxt, a_script in zip(output_included_session_contexts, output_python_scripts)]
button_executor = JupyterButtonRowWidget(button_defns=button_defns)

In [ ]:
reveal_in_system_file_manager(output_python_scripts[5])


## Working on redirecting stdout and print statements to file for batch scripts:
https://code.activestate.com/recipes/119404/

In [ ]:
# import builtins as __builtin__
# __builtins__

In [ ]:
from pyphocorehelpers.programming_helpers import override_function_context

import builtins

# def custom_print(*args, **kwargs):
# 	builtins.print(f'custom_print(...)\n\t')
# 	builtins.print(*args, **kwargs)

# def custom_print(*args, **kwargs):
# 	builtins.print(f'custom_print(...)\n\t')
# 	builtins.print(*args, **kwargs)
	
# with override_function_context(builtins, "print", custom_print):
# 	print(5)
# 	print('hi!') # RecursionError: maximum recursion depth exceeded


def test_fn():
	def custom_print(*args, **kwargs):
		builtins.print(f'custom_print(...)\n\t')
		builtins.print(*args, **kwargs)
		

	custom_print(5)
	custom_print('hi!') # RecursionError: maximum recursion depth exceeded


test_fn()


In [ ]:
__builtins__.print(f'custom_print(...)\n\t')

In [ ]:
from contextlib import redirect_stdout

# https://docs.python.org/3/library/contextlib.html#contextlib.redirect_stdout

with open('help.txt', 'w') as f:
    with redirect_stdout(f):
		print(5)
		print('hi!') # RecursionError: maximum recursion depth exceeded

In [ ]:
curr_task_logger = build_batch_task_logger(session_context=curr_session_context) # create logger , file_logging_dir=
_line_sweep = '=========================='
## REPLACES THE `print` function within this scope
def new_print(*args):
	# Call both regular print and logger.info
	print(*args)
	curr_task_logger.info(*args)


In [ ]:
output_python_scripts[5]

In [6]:
# %pdb on

# multiprocessing_kwargs = dict(use_multiprocessing=False, num_processes=1)
multiprocessing_kwargs = dict(use_multiprocessing=True, num_processes=3)

enable_full_pipeline_in_ram = False
# enable_full_pipeline_in_ram = True

# Whether to output figures:
should_perform_figure_generation_to_file=False
# should_perform_figure_generation_to_file=True

## Included Session Contexts:
# included_session_contexts = batch_progress_df[np.logical_and(batch_progress_df['has_user_replay_annotations'], batch_progress_df['is_ready'])]['context'].to_numpy().tolist()

# Only require sessions to have replay annotations:
# included_session_contexts = batch_progress_df[batch_progress_df['has_user_replay_annotations']]['context'].to_numpy().tolist()

# included_session_contexts = batch_progress_df['context'].to_numpy().tolist()[:4] # Only get the first 6
## Limit the contexts to run to the last N:
# included_session_contexts = included_session_contexts[3:6]

# ALL
included_session_contexts = included_session_contexts

# ## No filtering the sessions:
# included_session_contexts = None

if included_session_contexts is not None:
    print(f'len(included_session_contexts): {len(included_session_contexts)}')
else:
    print(f'included_session_contexts is None so all session contexts will be included.')

# included_session_contexts


# No Reloading
result_handler = BatchSessionCompletionHandler(force_reload_all=False,
                                                session_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
                                                global_computations_options=BatchComputationProcessOptions(should_load=True, should_compute=True, should_save=SavingOptions.IF_CHANGED),
                                                should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, should_generate_all_plots=True, saving_mode=PipelineSavingScheme.SKIP_SAVING, force_global_recompute=False,
                                                enable_full_pipeline_in_ram=enable_full_pipeline_in_ram,
                                                **multiprocessing_kwargs)


# # Forced Reloading:
# result_handler = BatchSessionCompletionHandler(force_reload_all=True,
#                                                 session_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 global_computations_options=BatchComputationProcessOptions(should_load=False, should_compute=True, should_save=SavingOptions.ALWAYS),
#                                                 should_perform_figure_generation_to_file=should_perform_figure_generation_to_file, saving_mode=PipelineSavingScheme.OVERWRITE_IN_PLACE, force_global_recompute=True,
#                                                 **multiprocessing_kwargs)


active_post_run_callback_fn = result_handler.on_complete_success_execution_session
# active_post_run_callback_fn = _temp_on_complete_success_execution_session



## Execute with the custom arguments.
active_computation_functions_name_includelist=['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'pf_dt_sequential_surprise', 'ratemap_peaks_prominence2d', 'position_decoding', 'position_decoding_two_step', 
                                               'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping',
                                                # 'long_short_inst_spike_rate_groups',
                                                'pf_dt_sequential_surprise', 'long_short_endcap_analysis']


# active_computation_functions_name_includelist=['_perform_baseline_placefield_computation']
global_batch_run.execute_all(force_reload=result_handler.force_reload_all, saving_mode=result_handler.saving_mode, skip_extended_batch_computations=True, post_run_callback_fn=active_post_run_callback_fn,
                             fail_on_exception=True, included_session_contexts=included_session_contexts,
                                                                                        **{'computation_functions_name_includelist': active_computation_functions_name_includelist,
                                                                                            'active_session_computation_configs': None,
                                                                                            'allow_processing_previously_completed': True}, **multiprocessing_kwargs) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 4m 39.8s

len(included_session_contexts): 15
Beginning processing with len(included_session_contexts): 15
build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-12_15-55-31"):build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-08_14-26-15"):build_batch_task_logger(module_name="LNX00052.kdiba.gor01.one.2006-6-09_1-22-43"):


	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-08_14-26-15 has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-08_14-26-15.log	 Batch Task logger com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-09_1-22-43 has file logging enabled and will log to EXTERNAL/TESTING/Logging/debug_com.PhoHale.PhoPy3DPositionAnalyis.Batch.runBatch.run_specific_batch.LNX00052.kdiba.gor01.one.2006-6-09_1-22-43.log	 Batch Task 

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.891
  warn( f"n_contours is 0 for level: {levii}")


done.
Loading pickled pipeline success: /media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/loadedSessPickle.pkl.
properties already present in pickled version. No need to save.
pipeline load success!
using provided computation_functions_name_includelist: ['pf_computation', 'pfdt_computation', 'firing_rate_trends', 'pf_dt_sequential_surprise', 'ratemap_peaks_prominence2d', 'position_decoding', 'position_decoding_two_step', 'long_short_decoding_analyses', 'jonathan_firing_rate_analysis', 'long_short_fr_indicies_analyses', 'short_long_pf_overlap_analyses', 'long_short_post_decoding', 'long_short_rate_remapping', 'pf_dt_sequential_surprise', 'long_short_endcap_analysis']
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will prevent recomputation even when the excludelist/includelist or computation function definitions change. Rework so that this is smarter.
	 TODO: this will

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.891
  warn( f"n_contours is 0 for level: {levii}")
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.495
  warn( f"n_contours is 0 for level: {levii}")


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.891000503675347
  warn( f"n_contours is 0 for level: {levii}")


...

/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.8910000898043593
  warn( f"n_contours is 0 for level: {levii}")
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:417: UserWarning: n_contours is 0 for level: 0.4950000498913107
  warn( f"n_contours is 0 for level: {levii}")


/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/PlacefieldDensityAnalysisComputationFunctions.py:525: UserWarning: 	WARNING: len(matching_vertical_scan_y_idxs) == 1: missing lower extrema, adding ybin_outer_extrema[0] = -1 to matching_vertical_scan_y_idxs
  warn(f'\tWARNING: len(matching_vertical_scan_y_idxs) == 1: missing lower extrema, adding ybin_outer_extrema[0] = {ybin_outer_extrema[0]} to matching_vertical_scan_y_idxs')


	 done.
extended_stats, maze already computed.
pf_dt_sequential_surprise missing.
	 Recomputing pf_dt_sequential_surprise...
Performing run_specific_computations_single_context on filtered_session with filter named "maze1"...
Performing run_specific_computations_single_context on filtered_session with filter named "maze2"...
Performing run_specific_computations_single_context on filtered_session with filter named "maze"...
	 done.
firing_rate_trends, maze already computed.
long_short_decoding_analyses, maze already computed.
short_long_pf_overlap_analyses, maze already computed.
long_short_fr_indicies_analyses, maze already computed.
jonathan_firing_rate_analysis, maze already computed.
long_short_post_decoding, maze already computed.
long_short_endcap_analysis, maze already computed.
done with all batch_extended_computations(...).
newly_computed_values: [('ratemap_peaks_prominence2d', 'maze'), ('pf_dt_sequential_surprise', 'maze')]. Saving global results...
global_computation_results_

/home/halechr/repos/Spike3D/.venv/lib/python3.9/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: 'fet11-01_12-58-54'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/halechr/repos/pyPhoPlaceCellAnalysis/src/pyphoplacecellanalysis/General/Pipeline/Stages/ComputationFunctions/MultiContextComputationFunctions/LongShortTrackComputations.py:231: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block3_values] [items->Index(['firing_rates', 'is_neuron_active', 'active_aclus'], dtype='object')]

  self.rdf.rdf.to_hdf(file_path, key=f'{key}/rdf/df') # , format='table', data_columns=True Can't do 'table' format because `TypeError: Cannot serialize the column [firing_rates] because its data 

long_short_inst_spike_rate_groups is missing and will be skipped
a_field: Flat_epoch_time_bins_mean
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/expected_v_observed_result/Flat_epoch_time_bins_mean
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: Flat_all_epochs_computed_expected_cell_num_spikes_LONG
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/expected_v_observed_result/Flat_all_epochs_computed_expected_cell_num_spikes_LONG
	 field not custom serializable! a_field_attr.type: <class 'numpy.ndarray'>.
a_field: observed_from_expected_diff_ptp_LONG
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/expected_v_observed_result/observed_from_expected_diff_ptp_LONG
	 field not custom serializable! a_field_attr.type: <class 'numpy.ma.core.MaskedArray'>.
a_field: observed_from_expected_diff_mean_LONG
	a_field_key: /kdiba/pin01/one/fet11-01_12-58-54/global_computations/expected_v_observed_resul

In [7]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
batch_progress_df.batch_results.build_all_columns()
good_only_batch_progress_df.batch_results.build_all_columns()
batch_progress_df

,format_name,animal,exper_name,session_name,context,basedirs,status,errors,session_datetime,n_long_laps,n_long_replays,n_short_laps,n_short_replays,is_ready,global_computation_result_file,loaded_session_pickle_file,ripple_result_file,has_user_replay_annotations,has_user_grid_bin_bounds_annotations
0,kdiba,gor01,one,2006-6-07_11-26-53,kdiba_gor01_one_2006-6-07_11-26-53,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,SessionBatchProgress.NOT_STARTED,None,2006-06-07 11:26:53,0,0,0,0,False,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-07_11-2...,False,True
1,kdiba,gor01,one,2006-6-08_14-26-15,kdiba_gor01_one_2006-6-08_14-26-15,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,SessionBatchProgress.COMPLETED,None,2006-06-08 14:26:15,40,279,40,224,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-2...,True,True
2,kdiba,gor01,one,2006-6-09_1-22-43,kdiba_gor01_one_2006-6-09_1-22-43,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43,SessionBatchProgress.COMPLETED,None,2006-06-09 01:22:43,46,179,40,142,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22...,True,True
3,kdiba,gor01,one,2006-6-09_3-23-37,kdiba_gor01_one_2006-6-09_3-23-37,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23-37,SessionBatchProgress.NOT_STARTED,None,2006-06-09 03:23:37,0,0,0,0,False,,,/media/MAX/Data/KDIBA/gor01/one/2006-6-09_3-23...,False,True
4,kdiba,gor01,one,2006-6-12_15-55-31,kdiba_gor01_one_2006-6-12_15-55-31,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,SessionBatchProgress.COMPLETED,None,2006-06-12 15:55:31,40,37,34,55,True,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-5...,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,kdiba,pin01,one,fet11-04_21-20-3,kdiba_pin01_one_fet11-04_21-20-3,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20-3,SessionBatchProgress.NOT_STARTED,None,2009-11-04 21:20:03,0,0,0,0,False,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,/media/MAX/Data/KDIBA/pin01/one/fet11-04_21-20...,False,True
68,kdiba,pin01,one,redundant,kdiba_pin01_one_redundant,/media/MAX/Data/KDIBA/pin01/one/redundant,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
69,kdiba,pin01,one,showclus,kdiba_pin01_one_showclus,/media/MAX/Data/KDIBA/pin01/one/showclus,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False
70,kdiba,pin01,one,sleep,kdiba_pin01_one_sleep,/media/MAX/Data/KDIBA/pin01/one/sleep,SessionBatchProgress.NOT_STARTED,None,NaT,0,0,0,0,False,,,,False,False


# Across Sessions After Batching Complete

In [8]:
#### Undocumented: HDF5 files containing links to other external .h5 files. These work!

included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_h5_paths

[PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-08_14-26-15/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-07_16-40-19/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-08_21-16-25/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-09_22-24-40/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/gor01/two/2006-6-12_16-53-46/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/vvp01/one/2006-4-09_17-29-30/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/vvp01/one/2006-4-10_12-25-50/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/vvp01/two/2006-4-09_16-40-54/output/pipeline_results.h5'),
 PosixPath('/media/MAX/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pi

In [9]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionTables

AcrossSessionTables.save_out_to_combined_file(included_session_contexts, included_h5_paths)

failed for file path: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/jonathan_fr_analysis/neuron_replay_stats_df/table. wth exception HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /usr/local/hdf5/lib/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1990, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 614, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0

In [10]:

neuron_replay_stats_table = AcrossSessionTables.build_neuron_replay_stats_table(included_session_contexts, included_h5_paths)
neuron_replay_stats_table

failed for file path: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/jonathan_fr_analysis/neuron_replay_stats_df/table. wth exception HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /usr/local/hdf5/lib/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1990, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 614, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0

,format_name,animal,exper_name,session_name,long_pf_peak_x,short_pf_peak_x,track_membership,long_non_replay_mean,short_non_replay_mean,non_replay_diff,...,short_num_replays,neuron_type,is_long_peak_left_cap,is_long_peak_right_cap,is_long_peak_either_cap,LS_pf_peak_x_diff,aclu,session_uid,neuron_uid,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,212.160000,NaN,LEFT_ONLY,0.419960,0.114724,-0.305237,...,0,pyr,False,False,False,NaN,2,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|2,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,136.160000,198.160000,SHARED,0.275226,1.227759,0.952532,...,52,pyr,False,False,False,-62.0,3,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|3,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,NaN,178.160000,RIGHT_ONLY,0.117397,0.834271,0.716874,...,67,pyr,False,False,False,NaN,4,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|4,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,98.160000,202.160000,SHARED,2.090985,1.844885,-0.246099,...,72,pyr,False,False,False,-104.0,5,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|5,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,NaN,NaN,SHARED,26.129319,12.892497,-13.236821,...,216,intr,False,False,False,NaN,6,kdiba|gor01|one|2006-6-08_14-26-15,kdiba|gor01|one|2006-6-08_14-26-15|6,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
830,kdiba,pin01,one,fet11-01_12-58-54,179.403791,177.403791,SHARED,0.892852,1.027642,0.134789,...,39,pyr,False,False,False,2.0,28,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|28,2009-11-01 12:58:54
831,kdiba,pin01,one,fet11-01_12-58-54,59.403791,107.403791,SHARED,0.837484,0.766268,-0.071216,...,43,pyr,True,False,True,-48.0,29,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|29,2009-11-01 12:58:54
832,kdiba,pin01,one,fet11-01_12-58-54,119.403791,127.403791,SHARED,0.712330,0.577649,-0.134681,...,32,pyr,False,False,False,-8.0,30,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|30,2009-11-01 12:58:54
833,kdiba,pin01,one,fet11-01_12-58-54,217.403791,33.403791,SHARED,0.902182,1.016473,0.114291,...,35,pyr,False,False,False,184.0,31,kdiba|pin01|one|fet11-01_12-58-54,kdiba|pin01|one|fet11-01_12-58-54|31,2009-11-01 12:58:54


In [11]:
long_short_fr_indicies_analysis_table = AcrossSessionTables.build_long_short_fr_indicies_analysis_table(included_session_contexts, included_h5_paths)
long_short_fr_indicies_analysis_table

failed for file path: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/global_computations/long_short_fr_indicies_analysis/table. wth exception HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /usr/local/hdf5/lib/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1990, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 614, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0, stored_eof

,format_name,animal,exper_name,session_name,index,neuron_uid,session_uid,aclu,x_frs_index,y_frs_index,session_datetime
0,kdiba,gor01,one,2006-6-08_14-26-15,0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba|gor01|one|2006-6-08_14-26-15,2,0.947513,1.000000,2006-06-08 14:26:15
1,kdiba,gor01,one,2006-6-08_14-26-15,1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba|gor01|one|2006-6-08_14-26-15,3,-0.855006,-0.544194,2006-06-08 14:26:15
2,kdiba,gor01,one,2006-6-08_14-26-15,2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba|gor01|one|2006-6-08_14-26-15,4,-0.972594,-1.000000,2006-06-08 14:26:15
3,kdiba,gor01,one,2006-6-08_14-26-15,3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba|gor01|one|2006-6-08_14-26-15,5,0.133418,-0.032454,2006-06-08 14:26:15
4,kdiba,gor01,one,2006-6-08_14-26-15,4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba|gor01|one|2006-6-08_14-26-15,6,0.433462,0.454120,2006-06-08 14:26:15
...,...,...,...,...,...,...,...,...,...,...,...
830,kdiba,pin01,one,fet11-01_12-58-54,26,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba|pin01|one|fet11-01_12-58-54,28,-0.567445,-0.172839,2009-11-01 12:58:54
831,kdiba,pin01,one,fet11-01_12-58-54,27,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba|pin01|one|fet11-01_12-58-54,29,-0.176580,0.089972,2009-11-01 12:58:54
832,kdiba,pin01,one,fet11-01_12-58-54,28,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba|pin01|one|fet11-01_12-58-54,30,-0.219618,-0.072261,2009-11-01 12:58:54
833,kdiba,pin01,one,fet11-01_12-58-54,29,kdiba|pin01|one|fet11-01_12-58-54|31,kdiba|pin01|one|fet11-01_12-58-54,31,0.314634,0.160301,2009-11-01 12:58:54


In [12]:
neuron_identities_table = AcrossSessionTables.build_neuron_identities_table(included_session_contexts, included_h5_paths)
neuron_identities_table

failed for file path: /media/MAX/Data/KDIBA/gor01/one/2006-6-12_15-55-31/output/pipeline_results.h5, table_key: /kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table. wth exception HDF5 error back trace

  File "H5F.c", line 620, in H5Fopen
    unable to open file
  File "H5VLcallback.c", line 3501, in H5VL_file_open
    failed to iterate over available VOL connector plugins
  File "H5PLpath.c", line 578, in H5PL__path_table_iterate
    can't iterate over plugins in plugin path '(null)'
  File "H5PLpath.c", line 620, in H5PL__path_table_iterate_process_path
    can't open directory: /usr/local/hdf5/lib/plugin
  File "H5VLcallback.c", line 3351, in H5VL__file_open
    open failed
  File "H5VLnative_file.c", line 97, in H5VL__native_file_open
    unable to open file
  File "H5Fint.c", line 1990, in H5F_open
    unable to read superblock
  File "H5Fsuper.c", line 614, in H5F__super_read
    truncated file: eof = 96, sblock->base_addr = 0, stored_eof = 2048

End of HDF5 error back tr

,global_uid,session_uid,session_datetime,format_name,animal,exper_name,session_name,neuron_id,neuron_type,cluster_index,qclu,shank_index
0,kdiba|gor01|one|2006-6-08_14-26-15|2,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,2,pyr,6,2,1
1,kdiba|gor01|one|2006-6-08_14-26-15|3,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,3,pyr,9,4,1
2,kdiba|gor01|one|2006-6-08_14-26-15|4,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,4,pyr,10,4,1
3,kdiba|gor01|one|2006-6-08_14-26-15|5,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,5,pyr,11,2,1
4,kdiba|gor01|one|2006-6-08_14-26-15|6,kdiba|gor01|one|2006-6-08_14-26-15,2006-06-08 14:26:15,kdiba,gor01,one,2006-6-08_14-26-15,6,intr,12,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...
830,kdiba|pin01|one|fet11-01_12-58-54|28,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,28,pyr,10,2,3
831,kdiba|pin01|one|fet11-01_12-58-54|29,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,29,pyr,11,2,3
832,kdiba|pin01|one|fet11-01_12-58-54|30,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,30,pyr,14,2,3
833,kdiba|pin01|one|fet11-01_12-58-54|31,kdiba|pin01|one|fet11-01_12-58-54,2009-11-01 12:58:54,kdiba,pin01,one,fet11-01_12-58-54,31,pyr,3,4,4


In [13]:
## Save converted back to .h5 file, .csv file, and several others

across_session_outputs = {'output/across_session_results/neuron_identities_table': neuron_identities_table,
 'output/across_session_results/long_short_fr_indicies_analysis_table': long_short_fr_indicies_analysis_table,
'output/across_session_results/neuron_replay_stats_table': neuron_replay_stats_table}


for k, v in across_session_outputs.items():
    k = Path(k)
    a_name = k.name
    print(f'a_name: {a_name}')
    v.to_csv(k.with_suffix(suffix='.csv'))
    # v.to_hdf(k, key=f'/{a_name}')    



a_name: neuron_identities_table
a_name: long_short_fr_indicies_analysis_table
a_name: neuron_replay_stats_table


In [ ]:

from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=0)


In [ ]:
# Options
session_identifier_key: str = 'session_name'
# session_identifier_key: str = 'session_datetime'

## !IMPORTANT! Count of the fields of interest using .value_counts(...) and converting to an explicit pd.DataFrame:
_out_value_counts_df: pd.DataFrame = neuron_replay_stats_table.value_counts(subset=['format_name', 'animal', 'session_name', 'session_datetime','track_membership'], normalize=False, sort=False, ascending=True, dropna=True).reset_index()
_out_value_counts_df.columns = ['format_name', 'animal', 'session_name', 'session_datetime', 'track_membership', 'count']
_out_value_counts_df

In [ ]:
## Find the time of the first session for each animal:
first_session_time  = _out_value_counts_df.groupby(['animal']).agg(session_datetime_first=('session_datetime', 'first')).reset_index()

## Subtract this initial time from all of the 'session_datetime' entries for each animal:
# Merge the first session time back into the original DataFrame
merged_df = pd.merge(_out_value_counts_df, first_session_time, on='animal')

# Subtract this initial time from all of the 'session_datetime' entries for each animal
merged_df['time_since_first_session'] = merged_df['session_datetime'] - merged_df['session_datetime_first']

merged_df

In [ ]:
import matplotlib.pyplot as plt

point_size = 8
df = _out_value_counts_df.copy()
animals = df['animal'].unique()
track_memberships = df['track_membership'].unique()

fig, axes = plt.subplots(1, len(animals), figsize=(15, 5))

for i, animal in enumerate(animals):
	ax = axes[i]
	subset_df = df[df['animal'] == animal]
	
	for track_membership in track_memberships:
		track_subset_df = subset_df[subset_df['track_membership'] == track_membership]
		ax.plot(track_subset_df['session_datetime'], track_subset_df['count'], label=f'Track: {track_membership}')
		ax.scatter(track_subset_df['session_datetime'], track_subset_df['count'], s=point_size)
		
	ax.set_title(f'Animal: {animal}')
	ax.set_xlabel('Session Datetime')
	ax.set_ylabel('Count')
	ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
_out_value_counts_df

In [ ]:


## See if the number of cells decreases over re-exposures to the track
df = _out_value_counts_df[_out_value_counts_df['animal'] == 'gor01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'pin01']
# df = _out_value_counts_df[_out_value_counts_df['animal'] == 'vvp01']

# Sort by column: 'session_datetime' (ascending)
df = df.sort_values(['session_datetime'])

'LEFT_ONLY'

# df.to_clipboard(index=False)
df

In [ ]:
## Get the number of cells in each session of the animal:
num_LxCs = df[df['track_membership'] == 'LEFT_ONLY']['count'].to_numpy()
num_Shared = df[df['track_membership'] == 'SHARED']['count'].to_numpy()
num_SxCs = df[df['track_membership'] == 'RIGHT_ONLY']['count'].to_numpy()

num_TotalCs = num_LxCs + num_Shared + num_SxCs
num_TotalCs

In [ ]:
# The only safe point to align each session to is the switchpoint (the delta):


In [ ]:
# Each session can be expressed in terms of time from the start of the first session.


In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure()
ax0 = fig.add_axes()
ax0.plot(

fig.show()


In [ ]:
df.plot()


In [ ]:
long_short_fr_indicies_analysis_table = AcrossSessionTables.build_long_short_fr_indicies_analysis_table(included_session_contexts, included_h5_paths)
long_short_fr_indicies_analysis_table

In [ ]:
neuron_identities_table = AcrossSessionTables.build_neuron_identities_table(included_session_contexts, included_h5_paths)
neuron_identities_table

In [ ]:
neuron_replay_stats_table = AcrossSessionTables.build_neuron_replay_stats_table(included_session_contexts, included_h5_paths)
neuron_replay_stats_table

In [ ]:
neuron_replay_stats_table

In [ ]:

from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsVisualizations

matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
graphics_output_dict = AcrossSessionsVisualizations.across_sessions_firing_rate_index_figure(long_short_fr_indicies_analysis_results=long_short_fr_indicies_analysis_table, num_sessions=0)

### Alluvial Flow Diagrams with `floweaver`

In [ ]:
import floweaver as fw

# How many of each track_membership type in each session?

# TAKES `_out_value_counts_df`
df = deepcopy(_out_value_counts_df)

## Make a floweaver Dataset out of it
dataset = fw.Dataset(df)

partition_animal = dataset.partition('animal') # equivalent to `fw.Partition.Simple('animal', np.unique(_out_value_counts_df['animal']))`
partition_session = dataset.partition(session_identifier_key)
partition_track_membership = dataset.partition('track_membership')

nodes = {
    'animal': fw.ProcessGroup(['animal'], partition_animal, title='Animal'), # fw.ProcessGroup(['gor01', 'pin01', 'vvp01']
    'track': fw.ProcessGroup(['track_membership'], partition_track_membership, title='TrackXclusivity'), # fw.ProcessGroup(['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED'], title='TrackXclusivity')
}

ordering = [
    ['animal'],  # put "animal" on the left...
    ['track'],   # ... and "track" on the right.
]

bundles = [
    fw.Bundle('animal', 'track'),
]

# partitions = [
#     fw.Partition.Simple('process', ['gor01', 'pin01', 'vvp01']),
#     fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']),
# ]

# nodes['animal'].partition = partitions[0]
# nodes['track'].partition = partitions[1]

sdd = fw.SankeyDefinition(nodes, bundles, ordering) # fw.SankeyDefinition(nodes, bundles, ordering, flow_partition=fw.Partition.Simple('process', []))


## This is key where the ['source', 'target', 'value'] come in:
# flow_data = df.groupby(['animal', 'track_membership'])['count'].sum().reset_index()
# flow_data.columns = ['source', 'target', 'value']

size_options = dict(width=500, height=400, margins=dict(left=100, right=100))
# fw.weave(sdd, dataset, palette='Set2_8').to_widget().auto_save_png('alluvial_diagram.png')
fw.weave(sdd, dataset, measures='count').to_widget(**size_options)



In [ ]:
## This is key where the ['source', 'target', 'value'] come in:
flow_data = df.groupby(['animal', 'track_membership'])['count'].sum().reset_index()
flow_data
# flow_data.columns = ['source', 'target', 'value']

In [ ]:
import floweaver as fw


df = deepcopy(_out_value_counts_df)

nodes = {
    'animal': fw.ProcessGroup(['gor01', 'pin01', 'vvp01'], title='Animal'),
    'track': fw.ProcessGroup(['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED'], title='TrackXclusivity'),
}

ordering = [
    ['animal'],  # put "animal" on the left...
    ['track'],   # ... and "track" on the right.
]

bundles = [
    fw.Bundle('animal', 'track'),
]

partitions = [
    fw.Partition.Simple('process', ['gor01', 'pin01', 'vvp01']),
    fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']),
]

nodes['animal'].partition = partitions[0]
nodes['track'].partition = partitions[1]
sdd = fw.SankeyDefinition(nodes, bundles, ordering, flow_partition=fw.Partition.Simple('process', []))

flow_data = df.groupby(['animal', 'track_membership'])['count'].sum().reset_index()
flow_data.columns = ['source', 'target', 'value']
fw.weave(sdd, flow_data, palette='Set2_8').to_widget().auto_save_png('alluvial_diagram.png')


#### Three-item flow:

In [ ]:

import floweaver as fw



def plot_alluvial_diagram(df: pd.DataFrame, session_identifier_key:str='session_datetime'):
	""" 2023-08-24 - Renders a flow diagram showing the breakdown of LxC, SxC, Shared neurons in each session for each animal. 

	Usage:
		alluvial_widget, sdd, flow_data = plot_alluvial_diagram(_out_value_counts_df, session_identifier_key=session_identifier_key)
		alluvial_widget

	"""
	df = df.copy()
	
	## Somehow I want to get a 'type' key through this process and into the final flow_data DataFrame so that I can color based on it as mentioned in this demo: https://notebooks.gesis.org/binder/jupyter/user/ricklupton-floweaver-586o9ybx/notebooks/docs/tutorials/quickstart.ipynb#
	
	# Define the flow from "animal" to "session_datetime"
	flow_animal_to_session = df.groupby(['animal', session_identifier_key])['count'].sum().reset_index()
	flow_animal_to_session.columns = ['source', 'target', 'value']

	# Define the flow from "session_datetime" to "track_membership"
	flow_session_to_track = df.groupby([session_identifier_key, 'track_membership'])['count'].sum().reset_index()
	flow_session_to_track.columns = ['source', 'target', 'value']

	# Combine the two flows
	flow_data: pd.DataFrame = pd.concat([flow_animal_to_session, flow_session_to_track], ignore_index=True)

	# Define the unique session identifiers:
	unique_session_identifiers = df[session_identifier_key].unique().tolist()

	nodes = {
		'animal': fw.ProcessGroup(['gor01', 'pin01', 'vvp01'], title='Animal'),
		'session': fw.ProcessGroup(unique_session_identifiers, title=session_identifier_key),  # New node for session_datetime
		'track': fw.ProcessGroup(['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED'], title='TrackXclusivity'),
	}

	ordering = [
		['animal'],
		['session'],  # Include session_datetime node
		['track'],
	]

	bundles = [
		fw.Bundle('animal', 'session'),  # Bundle from animal to session
		fw.Bundle('session', 'track'),   # Bundle from session to track
	]

	partitions = [
		fw.Partition.Simple('process', ['gor01', 'pin01', 'vvp01']),
		fw.Partition.Simple('process', unique_session_identifiers),  # Partition for session_datetime
		fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']),
	]

	nodes['animal'].partition = partitions[0]
	nodes['session'].partition = partitions[1]  # Assign partition for session_datetime
	nodes['track'].partition = partitions[2]
	
	flow_partition = fw.Partition.Simple('process', [])
	#flow_partition = fw.Partition.Simple('process', ['LEFT_ONLY', 'RIGHT_ONLY', 'SHARED']) # partitions[2]
	sdd = fw.SankeyDefinition(nodes, bundles, ordering, flow_partition=flow_partition) # 

	## Weave the plot:
	_out_widget = fw.weave(sdd, flow_data).to_widget().auto_save_png('alluvial_diagram_with_session.png')

	return _out_widget, sdd, flow_data



alluvial_widget, sdd, flow_data = plot_alluvial_diagram(_out_value_counts_df, session_identifier_key=session_identifier_key)
alluvial_widget

In [ ]:
included_h5_paths = [a_dir.joinpath('output','pipeline_results.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']]
included_global_computation_h5_paths = [a_dir.joinpath('output','global_computations.h5').resolve() for a_dir in included_session_batch_progress_df['basedirs']] 
included_h5_paths

In [ ]:
## 2023-08-10
# Add output modes for .pkl and .h5 files similar to figures. For GreatLakes Batch runs, allow output to the run folder.

# like the figures, file naming conventions should change for the different types.


In [ ]:
import seaborn as sns
from pyphocorehelpers.indexing_helpers import partition, safe_pandas_get_group
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import InstantaneousFiringRatesDataframeAccessor, AcrossSessionsVisualizations
from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import SingleBarResult

common_file_path = Path('output/active_across_session_scatter_plot_results.h5')
print(f'common_file_path: {common_file_path}')

# InstantaneousSpikeRateGroupsComputation, : pd.DataFrame
_shell_obj, loaded_result_df = InstantaneousFiringRatesDataframeAccessor.load_and_prepare_for_plot(common_file_path)
# Perform the actual plotting:
AcrossSessionsVisualizations.across_sessions_bar_graphs(_shell_obj, num_sessions=13, save_figure=False, enable_tiny_point_labels=False, enable_hover_labels=False)

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
sessions_with_results

In [ ]:
session_identifiers = included_session_contexts


In [ ]:
global_batch_run

In [ ]:
# Save to pickle:
saveData(global_batch_result_file_path, global_batch_run) # Update the global batch run dictionary

In [ ]:
# Combine across .h5 files:
import tables as tb
import h5py
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import H5ExternalLinkBuilder

# f'/kdiba/gor01/one/2006-6-12_15-55-31/neuron_identities/table'

session_identifiers, pkl_output_paths, hdf5_output_paths

file_path = f'output/test_external_links.h5'
a_global_computations_group_key: str = f"{session_group_key}/global_computations"
with tb.open_file(file_path, mode='w') as f: # this mode='w' is correct because it should overwrite the previous file and not append to it.
    a_global_computations_group = f.create_group(session_group_key, 'global_computations', title='the result of computations that operate over many or all of the filters in the session.', createparents=True)
    an_external_link = f.create_external_link(f'', n

## Extract output files from all completed sessions:

In [ ]:
# '/nfs/turbo/umms-kdiba/Data/KDIBA/gor01/one/2006-6-09_1-22-43/output/pipeline_results.h5'

# kdiba_vvp01_two_2006-4-10_12-58-3
# 	outputs_local ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/loadedSessPickle.pkl')}
# 	outputs_global ={'pkl': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/global_computation_results.pkl'), 'hdf5': PosixPath('/nfs/turbo/umms-kdiba/Data/KDIBA/vvp01/two/2006-4-10_12-58-3/output/pipeline_results.h5')}

session_identifiers, pkl_output_paths, hdf5_output_paths = global_batch_run.build_output_files_lists()

In [ ]:
session_identifiers

In [ ]:
pkl_output_paths

In [ ]:
hdf5_output_paths

In [ ]:
# Save output filelist:


def save_filelist_to_text_file(hdf5_output_paths, filelist_path: Path):
    _out_string = '\n'.join([str(a_file) for a_file in hdf5_output_paths])
    print(f'{_out_string}')
    print(f'saving out to "{filelist_path}"...')
    with open(filelist_path, 'w') as f:
        f.write(_out_string)
    return _out_string, filelist_path


h5_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_HDF5_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_HDF5_savepath = save_filelist_to_text_file(hdf5_output_paths, h5_filelist_path)

pkls_filelist_path = global_data_root_parent_path.joinpath(f'fileList_Greatlakes_pkls_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, src_filelist_pkls_savepath = save_filelist_to_text_file(pkl_output_paths, pkls_filelist_path)


In [ ]:
from pyphocorehelpers.Filesystem.path_helpers import convert_filelist_to_new_parent
# source_parent_path = Path(r'/media/MAX/cloud/turbo/Data')
source_parent_path = Path(r'/nfs/turbo/umms-kdiba/Data')
dest_parent_path = Path(r'/~/W/Data/')
# # Build the destination filelist from the source_filelist and the two paths:
filelist_source = hdf5_output_paths
filelist_dest_paths = convert_filelist_to_new_parent(filelist_source, original_parent_path=source_parent_path, dest_parent_path=dest_parent_path)
filelist_dest_paths

dest_Apogee_h5_filelist_path = global_data_root_parent_path.joinpath(f'dest_fileList_Apogee_{BATCH_DATE_TO_USE}.txt').resolve()
_out_string, dest_filelist_savepath = save_filelist_to_text_file(filelist_dest_paths, dest_Apogee_h5_filelist_path)

In [ ]:
from pyphoplacecellanalysis.General.Batch.runBatch import PipelineCompletionResult
from neuropy.core.epoch import Epoch

# Save to HDF5
suffix = f'{BATCH_DATE_TO_USE}'
## Build Pickle Path:
file_path = global_data_root_parent_path.joinpath(f'global_batch_output_{suffix}.h5').resolve()
file_path

In [ ]:
global_batch_run.to_hdf(file_path,'/')

In [ ]:
# Get only the sessions with non-None results
sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]

# list(global_batch_run.session_batch_outputs.keys())

# Somewhere in there there are `InstantaneousSpikeRateGroupsComputation` results to extract
across_sessions_instantaneous_fr_dict = {} # InstantaneousSpikeRateGroupsComputation

# good_session_batch_outputs = global_batch_run.session_batch_outputs

sessions_with_results = [a_ctxt for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None]
good_session_batch_outputs = {a_ctxt:a_result for a_ctxt, a_result in global_batch_run.session_batch_outputs.items() if a_result is not None}

for a_ctxt, a_result in good_session_batch_outputs.items():
    if a_result is not None:
        # a_good_result = a_result.__dict__.get('across_sessions_batch_results', {}).get('inst_fr_comps', None)
        a_good_result = a_result.across_session_results.get('inst_fr_comps', None)
        if a_good_result is not None:
            across_sessions_instantaneous_fr_dict[a_ctxt] = a_good_result
            # print(a_result['across_sessions_batch_results']['inst_fr_comps'])
            
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

# When done, `result_handler.across_sessions_instantaneous_fr_dict` is now equivalent to what it would have been before. It can be saved using the normal `.save_across_sessions_data(...)`

## Save the instantaneous firing rate results dict: (# Dict[IdentifyingContext] = InstantaneousSpikeRateGroupsComputation)
AcrossSessionsResults.save_across_sessions_data(across_sessions_instantaneous_fr_dict=across_sessions_instantaneous_fr_dict, global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl')

# ## Save pickle:
# inst_fr_output_filename=f'across_session_result_long_short_inst_firing_rate_{BATCH_DATE_TO_USE}.pkl'
# global_batch_result_inst_fr_file_path = Path(global_data_root_parent_path).joinpath(inst_fr_output_filename).resolve() # Use Default
# print(f'global_batch_result_inst_fr_file_path: {global_batch_result_inst_fr_file_path}')
# # Save the all sessions instantaneous firing rate dict to the path:
# saveData(global_batch_result_inst_fr_file_path, across_sessions_instantaneous_fr_dict)

In [ ]:
across_sessions_instantaneous_fr_dict

In [ ]:
[a_ctxt.get_initialization_code_string() for a_ctxt in sessions_with_results]

# OLD

## 2023-07-14 - Load Saved across-sessions-data and testing Batch-computed inst_firing_rates:

In [ ]:
# from neuropy.utils.matplotlib_helpers import matplotlib_configuration_update
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import PaperFigureTwo, InstantaneousSpikeRateGroupsComputation
# from pyphoplacecellanalysis.General.Pipeline.Stages.ComputationFunctions.SpikeAnalysis import SpikeRateTrends
# from pyphoplacecellanalysis.General.Batch.PhoDiba2023Paper import list_of_dicts_to_dict_of_lists
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, AcrossSessionsVisualizations

## Load the saved across-session results:
# inst_fr_output_filename = 'long_short_inst_firing_rate_result_handlers_2023-07-12.pkl'
# inst_fr_output_filename = 'across_session_result_long_short_inst_firing_rate.pkl'
# inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-07-21.pkl'
# inst_fr_output_filename=f'across_session_result_handler_{BATCH_DATE_TO_USE}.pkl'
inst_fr_output_filename='across_session_result_long_short_inst_firing_rate_2023-08-09_Test.pkl'
across_session_inst_fr_computation, across_sessions_instantaneous_fr_dict, across_sessions_instantaneous_frs_list = AcrossSessionsResults.load_across_sessions_data(global_data_root_parent_path=global_data_root_parent_path, inst_fr_output_filename=inst_fr_output_filename)
# across_sessions_instantaneous_fr_dict = loadData(global_batch_result_inst_fr_file_path)
num_sessions = len(across_sessions_instantaneous_fr_dict)
print(f'num_sessions: {num_sessions}')

In [ ]:
across_sessions_instantaneous_frs_list

In [ ]:
## Hacks the `PaperFigureTwo` and `InstantaneousSpikeRateGroupsComputation` 
global_multi_session_context, _out_aggregate_fig_2 = AcrossSessionsVisualizations.across_sessions_bar_graphs(across_session_inst_fr_computation, num_sessions, enable_tiny_point_labels=False, enable_hover_labels=False)


In [ ]:
across_session_inst_fr_computation.LxC_scatter_props
across_session_inst_fr_computation.SxC_scatter_props

In [ ]:
from pyphoplacecellanalysis.General.Batch.AcrossSessionResults import AcrossSessionsResults, InstantaneousFiringRatesDataframeAccessor, InstantaneousSpikeRateGroupsComputation, trackMembershipTypesEnum, trackExclusiveToMembershipTypeDict, trackExclusiveToMembershipTypeReverseDict

## Specify the output file:
common_file_path = Path('output/test_across_session_scatter_plot_new.h5')
print(f'common_file_path: {common_file_path}')
InstantaneousFiringRatesDataframeAccessor.add_results_to_inst_fr_results_table(curr_active_pipeline, common_file_path, file_mode='a')

In [ ]:
# Build the unique scatter plot dictionaries:
across_session_contexts = list(across_sessions_instantaneous_fr_dict.keys())
unique_animals = IdentifyingContext.find_unique_values(across_session_contexts)['animal'] # {'gor01', 'pin01', 'vvp01'}
# Get number of animals to plot
marker_list = [(5, i) for i in np.arange(len(unique_animals))] # [(5, 0), (5, 1), (5, 2)]
scatter_props = [{'marker': mkr} for mkr in marker_list]  # Example, you should provide your own scatter properties
scatter_props_dict = dict(zip(unique_animals, scatter_props))
# {'pin01': {'marker': (5, 0)},
#  'gor01': {'marker': (5, 1)},
#  'vvp01': {'marker': (5, 2)}}
scatter_props_dict

# Pass a function that will return a set of kwargs for a given context
def _return_scatter_props_fn(ctxt: IdentifyingContext):
	""" captures `scatter_props_dict` """
	animal_id = str(ctxt.animal)
	return scatter_props_dict[animal_id]
	


In [ ]:
## Aggregate across all of the sessions to build a new combined `InstantaneousSpikeRateGroupsComputation`, which can be used to plot the "PaperFigureTwo", bar plots for many sessions.
global_multi_session_context = IdentifyingContext(format_name='kdiba', num_sessions=num_sessions) # some global context across all of the sessions, not sure what to put here.

# To correctly aggregate results across sessions, it only makes sense to combine entries at the `.cell_agg_inst_fr_list` variable and lower (as the number of cells can be added across sessions, treated as unique for each session).

## Display the aggregate across sessions:
_out_fig_2 = PaperFigureTwo(instantaneous_time_bin_size_seconds=0.01) # WARNING: we didn't save this info
_out_fig_2.computation_result = across_session_inst_fr_computation # the result loaded from the file
_out_fig_2.active_identifying_session_ctx = across_session_inst_fr_computation.active_identifying_session_ctx
# Set callback, the only self-specific property
# _out_fig_2._pipeline_file_callback_fn = curr_active_pipeline.output_figure # lambda args, kwargs: self.write_to_file(args, kwargs, curr_active_pipeline)
_out_fig_2.scatter_props_fn = _return_scatter_props_fn

In [ ]:
LxC_aclus = _out_fig_2.computation_result.LxC_aclus
SxC_aclus = _out_fig_2.computation_result.SxC_aclus

LxC_aclus

In [ ]:
from pyphoplacecellanalysis.General.Mixins.ExportHelpers import FigureOutputManager, FigureOutputLocation, ContextToPathMode

registered_output_files = {}

def output_figure(final_context: IdentifyingContext, fig, write_vector_format:bool=False, write_png:bool=True, debug_print=True):
    """ outputs the figure using the provided context. """
    from pyphoplacecellanalysis.General.Mixins.ExportHelpers import build_and_write_to_file
    def register_output_file(output_path, output_metadata=None):
        """ registers a new output file for the pipeline """
        print(f'register_output_file(output_path: {output_path}, ...)')
        registered_output_files[output_path] = output_metadata or {}

    fig_out_man = FigureOutputManager(figure_output_location=FigureOutputLocation.DAILY_PROGRAMMATIC_OUTPUT_FOLDER, context_to_path_mode=ContextToPathMode.HIERARCHY_UNIQUE)
    active_out_figure_paths = build_and_write_to_file(fig, final_context, fig_out_man, write_vector_format=write_vector_format, write_png=write_png, register_output_file_fn=register_output_file)
    return active_out_figure_paths, final_context


# Set callback, the only self-specific property
_out_fig_2._pipeline_file_callback_fn = output_figure

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
_out_fig_2.computation_result.Fig2_Laps_FR

In [ ]:
# Showing
restore_previous_matplotlib_settings_callback = matplotlib_configuration_update(is_interactive=True, backend='Qt5Agg')
# Perform interactive Matplotlib operations with 'Qt5Agg' backend
_fig_2_theta_out, _fig_2_replay_out = _out_fig_2.display(active_context=global_multi_session_context, title_modifier_fn=lambda original_title: f"{original_title} ({num_sessions} sessions)", save_figure=True)
	
_out_fig_2.perform_save()

### Single Session testing:


In [ ]:
_test_out = global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, computation_functions_name_includelist =['_perform_baseline_placefield_computation'], active_session_computation_configs=None) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)
_test_out

# global_batch_run.execute_session(session_context=curr_sess_context, force_reload=True, skip_extended_batch_computations=True, **{'computation_functions_name_includelist': ['_perform_baseline_placefield_computation'], 'active_session_computation_configs': None}) # can override `active_session_computation_configs` if we want to set custom ones like only the laps.)

# 23.5s

In [ ]:

full_good_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is None]
bad_dirs = [k for k, v in global_batch_run.session_batch_errors.items() if v is not None]
full_good_dirs
bad_dirs

In [ ]:
global_batch_run.session_batch_status

In [ ]:
global_batch_run.session_batch_status
global_batch_run.session_batch_errors

## Get good sessions for use in the specific session processing notebook:

In [ ]:
batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=False) # all
good_only_batch_progress_df = global_batch_run.to_dataframe(expand_context=True, good_only=True)
good_only_batch_progress_df

In [ ]:
## Get the list of sessions that are completely ready to process:
full_good_ready_to_process_sessions = list(good_only_batch_progress_df['context'].to_numpy())
full_good_ready_to_process_sessions
# Get good sessions for use in the specific session processing notebook:

In [ ]:
run["good_sessions_list"].extend(full_good_ready_to_process_sessions)

In [ ]:
run.stop()
project.stop()

In [ ]:

print(",\n".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # List definitions

# [IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19'),
# IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46'),
# IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')]

In [ ]:
print("\ncurr_context = ".join([ctx.get_initialization_code_string() for ctx in full_good_ready_to_process_sessions])) # Line definitions

# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-08_14-26-15')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-09_1-22-43')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-12_15-55-31')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='one',session_name='2006-6-13_14-42-6')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-07_16-40-19')
# curr_context = IdentifyingContext(format_name='kdiba',animal='gor01',exper_name='two',session_name='2006-6-12_16-53-46')
# curr_context = IdentifyingContext(format_name='kdiba',animal='vvp01',exper_name='one',session_name='2006-4-09_17-29-30')

In [ ]:
good_only_batch_progress_df

In [ ]:
from datetime import datetime

# datetime object containing current date and time
save_time = datetime.now()
 
print("save_time =", save_time)

# dd/mm/YY H:M:S
dt_string = save_time.strftime("%Y-%m-%d_%I-%M%p")
print("date and time =", dt_string)

In [ ]:
## Get output file paths:
completed_pipeline_filename = 'loadedSessPickle.pkl'
completed_global_computations_filename = 'outputs/global_computation_results.pkl'

full_good_ready_to_process_session_paths = list(good_only_batch_progress_df['basedirs'].to_numpy())
session_paths_output_folders = [sess_path.joinpath('outputs').resolve() for sess_path in full_good_ready_to_process_session_paths]

completed_pipeline_file_paths = [sess_path.joinpath(completed_pipeline_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths = [sess_path.joinpath(completed_global_computations_filename).resolve() for sess_path in full_good_ready_to_process_session_paths]
completed_global_computations_file_paths

In [ ]:
# Countable Additivity 
# Any countable collections of points is size 0

